# Derivatives

This notebook was automatically generated from the Algorithms for Optimization source code. Each cell generates a figure from the original text. While this code is not optimized for use in lectures, we provide it here to be adapted for such projects. We hope you find it useful.

In [ ]:
include("support_code.jl");

In [ ]:
p = let

f = x->sin(x)

x₀ = 0.5
ymin = -1.1
lo = -2
hi = 3

function get_axis(h)
a, b = x₀, x₀ + h
fa, fb = f(a), f(b)
m = (fb - fa) / (b-a)

p = Plots.Plot[]
push!(p, Plots.Linear(f, (lo,hi), style="thick, black"))
push!(p, Plots.Linear([a,a],[fa, ymin], style="solid, black, mark=none"))
push!(p, Plots.Linear([b,b],[fb, ymin], style="solid, black, mark=none"))
push!(p, Plots.Linear([lo,hi],[ymin, ymin], style="solid, black, mark=none"))
push!(p, Plots.Linear([lo, hi],[fa + (lo-a)*m, fa + (hi-a)*m], style="ultra thick, solid, pastelBlue, mark=none"))
push!(p, Plots.Scatter([a, b],[fa, fb], style="solid, mark=*, mark options={draw=black, fill=black}"))

Axis(p, style="axis lines=left, axis line style = {transparent}, xtick={$(x₀ + h/2)}, xticklabels={\$h\$}, ytick=\\empty, xtick style={draw=none}, ytick style={draw=none}") #$
end

g = GroupPlot(3, 1, groupStyle="horizontal sep=1cm", style="ymin=-1.1, ymax=1.1, width=6cm")
push!(g, get_axis(2.0))
push!(g, get_axis(1.0))
push!(g, get_axis(0.5))
g
end

plot(p)

In [ ]:
diff_forward(f, x; h=sqrt(eps(Float64))) = (f(x+h) - f(x))/h
diff_central(f, x; h=cbrt(eps(Float64))) = (f(x+h/2) - f(x-h/2))/h
diff_backward(f, x; h=sqrt(eps(Float64))) = (f(x) - f(x-h))/h
diff_complex(f, x; h=1e-20) = imag(f(x + h*im)) / h

p = let
	x = 0.5
	d_true = 1.0

	arr_h = collect(10 .^ range(-17, stop=1, length=101))
	arr_forward = [abs(d_true - diff_forward(sin, x, h=h)/cos(x))
					for h in arr_h]
	arr_central = [abs(d_true - diff_central(sin, x, h=h)/cos(x))
					for h in arr_h]
	arr_complex = [abs(d_true - diff_complex(sin, x, h=h)/cos(x))
					for h in arr_h]

	Axis([
		Plots.Linear(arr_h, arr_complex, style="thick, solid,
			pastelGreen, mark=none", legendentry="complex"),
		Plots.Linear(arr_h, arr_forward, style="thick, solid,
			pastelBlue, mark=none", legendentry="forward"),
		Plots.Linear(arr_h, arr_central, style="thick, solid,
			pastelRed, mark=none", legendentry="central")],
		xmode="log", ymode="log",
		xlabel="step size \$h\$",
		ylabel="absolute relative error",
		width="10cm", height="8cm",
		style="legend pos=outer north east"
	)
end
plot(p)